#  Protein Binding Pocket Finder

This notebook implements a geometry-based algorithm for predicting **ligand binding pockets** in protein structures.

## Principle
Instead of computing expensive energy functions, the protein surface is sampled geometrically:
1. A 3D grid of points is placed over the entire protein
2. Points that are too close to the protein (collision) or too far away (empty space) are removed
3. The remaining points — located inside cavities and pockets — are grouped into clusters
4. Each cluster corresponds to one potential binding pocket

Binding pockets are often **evolutionarily conserved**: residues that form a functional site tend to be invariant across species. This is used as an additional filter after the geometric step.

## Input
A `.pdb` file (here: `1H8D.pdb` — a serine protease from the RCSB Protein Data Bank)

---

## 0. Imports

All required libraries are imported here in one central place:
- **`numpy`** – fast array operations and vector math
- **`Bio.PDB`** – Biopython module for reading and analyzing PDB files
- **`scipy.spatial.KDTree`** – efficient spatial lookups (much faster than naive distance calculations)
- **`sklearn.cluster.DBSCAN`** – density-based clustering algorithm that requires no fixed cluster count
- **`Bio.AlignIO`** – reading Multiple Sequence Alignments (MSA) for conservation scoring
- **`collections.Counter`** – frequency counting for conservation score calculation
- **`time`** – runtime measurement
- **`os`** – directory creation for export

In [269]:
import numpy as np
import time
import os

from Bio import PDB, AlignIO
from Bio.PDB import is_aa
from Bio.PDB.Polypeptide import is_aa
from scipy.spatial import KDTree
from sklearn.cluster import DBSCAN
from collections import Counter

---
## 1. Reading the Protein Structure
PDB files contain not only the protein atoms, but also water molecules, ligands, ions, and other hetero-atoms.

For binding site prediction, we only keep atoms belonging to the protein itself.

### Why filter?

* **Water molecules** introduce noise because their positions are often not biologically stable.
* **Ligands and co-factors** may already occupy binding pockets and would bias the prediction.

### Filtering in BioPython

The structure is parsed using BioPython and iterated hierarchically:

Structure → Model → Chain → Residue → Atom

We use:

```python
is_aa(residue, standard=True)
```

to select only the 20 standard amino acids.

This automatically excludes:

* water (HOH)
* ligands
* ions
* co-factors

### Example PDB records

```
ATOM      1  N   ALA A   1 ...
HETATM 1234  O   HOH A 201 ...
HETATM 2345 FE   HEM A 500 ...
```

Only `ATOM` records corresponding to amino acids are kept.

The resulting atom list is used for grid generation in the next step.


In [270]:
def get_protein_structure(pdb_file: str):
    """
    Reads a PDB file and returns the structure object along with a list
    of all protein atoms (excluding water and ligands).

    Parameters:
        pdb_file: Path to the .pdb file

    Returns:
        structure:     Bio.PDB Structure object (full hierarchy)
        protein_atoms: List of all Atom objects belonging to amino acids
    """
    # QUIET=True suppresses warnings for minor format inconsistencies
    # PERMISSIVE=True allows non-standard PDB entries
    parser = PDB.PDBParser(QUIET=True, PERMISSIVE=True)

    # Load the structure — 'protein_obj' is an internal label for the object
    structure = parser.get_structure('protein_obj', pdb_file)

    protein_atoms = []

    # Hierarchical iteration: Model → Chain → Residue → Atom
    for model in structure:
        for chain in model:
            for residue in chain:
                # robust protein check: only standard amino acids
                if not is_aa(residue, standard=True):
                    continue

                for atom in residue:
                    protein_atoms.append(atom)

    print(f"Loaded: {len(protein_atoms)} protein atoms found.")
    return structure, protein_atoms

---
## 1b. Saving a Cleaned PDB File

Raw PDB files often contain multiple chains, water molecules, and ligands that are not relevant for pocket prediction.
`save_clean_protein` creates a new, minimal PDB file containing **only the selected protein chain(s)** with standard amino acids.

### Why save a clean file?

* Reduces file size and complexity for downstream tools
* Ensures consistent chain labelling for all subsequent steps
* Avoids artefacts from co-crystallised ligands or water molecules that could bias the grid filtering

The function builds a fresh BioPython `Structure` object, copies only the desired chains and amino-acid residues,
and writes the result with `PDBIO`. Only residues passing `is_aa(standard=True)` are included.

In [271]:
from Bio.PDB import PDBIO
from Bio.PDB.Structure import Structure
from Bio.PDB.Model import Model
from Bio.PDB.Chain import Chain


def save_clean_protein(structure, output_file, protein_chains=["H"]):
    """
    Saves a cleaned PDB containing only the specified chains and
    standard amino acids (no water, ligands, or ions).

    Parameters:
        structure:      Bio.PDB Structure object (full, unfiltered)
        output_file:    Path for the output .pdb file
        protein_chains: List of chain IDs to retain (default: ['H'])
    """
    clean_structure = Structure("clean")
    model_new = Model(0)
    clean_structure.add(model_new)

    for model in structure:
        for chain in model:
            # Keep only the desired chains
            if chain.id not in protein_chains:
                continue

            new_chain = Chain(chain.id)

            for residue in chain:
                if is_aa(residue, standard=True):
                    new_chain.add(residue.copy())

            model_new.add(new_chain)

    io = PDBIO()
    io.set_structure(clean_structure)
    io.save(output_file)

    print("Clean protein saved:", output_file)

---
## 1c. Loading and Cleaning the Protein

This cell executes the loading and cleaning pipeline:

1. **Load original PDB** (`1H8D.pdb`) — reads all atoms including ligands, water, and multiple chains.
2. **Save cleaned PDB** (`1H8D_cleaned.pdb`) — retains only chain H with standard amino acids.
3. **Reload cleaned file** — `clean_atoms` is used by all subsequent steps.
4. **Verification printout** — lists every residue in the cleaned chain.

> **Note:** Chain `H` is specific to `1H8D`. Adjust `protein_chains` for other PDB entries.

In [272]:
FILE_PATH = "1H8D.pdb"

# Step 1: Load original structure (all chains, ligands, water)
structure, atoms = get_protein_structure(FILE_PATH)

# Step 2: Save cleaned file (chain H, amino acids only)
save_clean_protein(structure, "1H8D_cleaned.pdb", protein_chains=["H"])

# Step 3: Reload cleaned file — this is the input for all following steps
clean_structure, clean_atoms = get_protein_structure("1H8D_cleaned.pdb")

print(f"\nAtom count: original={len(atoms)}, cleaned={len(clean_atoms)}")
print(f"Removed: {len(atoms) - len(clean_atoms)} atoms (ligands, water, other chains)\n")

print("VERIFY CLEANED FILE:\n")
for model in clean_structure:
    for chain in model:
        print("CHAIN:", chain.id)
        for residue in chain:
            print(residue.get_resname())

Loaded: 2333 protein atoms found.
Clean protein saved: 1H8D_cleaned.pdb
Loaded: 2030 protein atoms found.

Atom count: original=2333, cleaned=2030
Removed: 303 atoms (ligands, water, other chains)

VERIFY CLEANED FILE:

CHAIN: H
ILE
VAL
GLU
GLY
SER
ASP
ALA
GLU
ILE
GLY
MET
SER
PRO
TRP
GLN
VAL
MET
LEU
PHE
ARG
LYS
SER
PRO
GLN
GLU
LEU
LEU
CYS
GLY
ALA
SER
LEU
ILE
SER
ASP
ARG
TRP
VAL
LEU
THR
ALA
ALA
HIS
CYS
LEU
LEU
TYR
PRO
PRO
TRP
ASP
LYS
ASN
PHE
THR
GLU
ASN
ASP
LEU
LEU
VAL
ARG
ILE
GLY
LYS
HIS
SER
ARG
THR
ARG
TYR
GLU
ARG
ASN
ILE
GLU
LYS
ILE
SER
MET
LEU
GLU
LYS
ILE
TYR
ILE
HIS
PRO
ARG
TYR
ASN
TRP
ARG
GLU
ASN
LEU
ASP
ARG
ASP
ILE
ALA
LEU
MET
LYS
LEU
LYS
LYS
PRO
VAL
ALA
PHE
SER
ASP
TYR
ILE
HIS
PRO
VAL
CYS
LEU
PRO
ASP
ARG
GLU
THR
ALA
ALA
SER
LEU
LEU
GLN
ALA
GLY
TYR
LYS
GLY
ARG
VAL
THR
GLY
TRP
GLY
ASN
LEU
LYS
GLU
THR
GLY
GLN
PRO
SER
VAL
LEU
GLN
VAL
VAL
ASN
LEU
PRO
ILE
VAL
GLU
ARG
PRO
VAL
CYS
LYS
ASP
SER
THR
ARG
ILE
ARG
ILE
THR
ASP
ASN
MET
PHE
CYS
ALA
GLY
TYR
LYS
PRO
ASP
GLU
GLY
LYS
ARG
GLY
ASP
ALA

---
## 1d. Verifying the Cleaned Structure

Before proceeding, we confirm that no hetero-atoms remain.
A blank hetero flag (`' '`) means the residue is a standard amino acid — the expected result.
Any `'H_'` prefix would indicate a remaining hetero-atom.

In [273]:
print("NON-PROTEIN RESIDUES IN CLEANED FILE:\n")
for model in clean_structure:
    for chain in model:
        for residue in chain:
            hetflag = residue.id[0]
            print(f"Chain: {chain.id} | Name: {residue.get_resname()} | Hetero flag: {hetflag!r}")

NON-PROTEIN RESIDUES IN CLEANED FILE:

Chain: H | Name: ILE | Hetero flag: ' '
Chain: H | Name: VAL | Hetero flag: ' '
Chain: H | Name: GLU | Hetero flag: ' '
Chain: H | Name: GLY | Hetero flag: ' '
Chain: H | Name: SER | Hetero flag: ' '
Chain: H | Name: ASP | Hetero flag: ' '
Chain: H | Name: ALA | Hetero flag: ' '
Chain: H | Name: GLU | Hetero flag: ' '
Chain: H | Name: ILE | Hetero flag: ' '
Chain: H | Name: GLY | Hetero flag: ' '
Chain: H | Name: MET | Hetero flag: ' '
Chain: H | Name: SER | Hetero flag: ' '
Chain: H | Name: PRO | Hetero flag: ' '
Chain: H | Name: TRP | Hetero flag: ' '
Chain: H | Name: GLN | Hetero flag: ' '
Chain: H | Name: VAL | Hetero flag: ' '
Chain: H | Name: MET | Hetero flag: ' '
Chain: H | Name: LEU | Hetero flag: ' '
Chain: H | Name: PHE | Hetero flag: ' '
Chain: H | Name: ARG | Hetero flag: ' '
Chain: H | Name: LYS | Hetero flag: ' '
Chain: H | Name: SER | Hetero flag: ' '
Chain: H | Name: PRO | Hetero flag: ' '
Chain: H | Name: GLN | Hetero flag: ' '
C

---
## 2. Creating the Search Grid

To search for potential binding sites, we place a regular 3D grid around the protein.

### Parameters

| Parameter | Meaning                      | Typical Value                  |
| --------- | ---------------------------- | ------------------------------ |
| `spacing` | Distance between grid points | 2.0 Å (testing), 1.0 Å (final) |
| `buffer`  | Extra space around protein   | 5.0 Å                          |

### Method

First, the bounding box of the protein is computed from the atom coordinates.
Then, evenly spaced points are generated along each axis using `np.arange()` and
combined into 3D coordinates using `np.meshgrid()`.

The result is a NumPy array of shape `(N, 3)` where each row is one grid point.

### Note on Performance

Smaller spacing increases resolution but dramatically increases the number of grid points
(~8× more when going from 2.0 Å to 1.0 Å).

In [274]:
def create_search_grid(protein_atoms: list, spacing: float = 1.0) -> np.ndarray:
    """
    Creates a uniform 3D grid around the protein.

    Parameters:
        protein_atoms: List of Biopython Atom objects
        spacing:       Distance between grid points in Ångström

    Returns:
        grid: numpy array of shape (N, 3) containing N grid points
    """
    BUFFER = 5.0  # Å — padding region extending beyond the protein

    # Extract all atom coordinates into a single (N, 3) matrix
    coords = np.array([atom.get_coord() for atom in protein_atoms])

    # Bounding box: smallest and largest coordinates along x, y, z
    min_coords = coords.min(axis=0) - BUFFER
    max_coords = coords.max(axis=0) + BUFFER

    # Generate axis arrays
    x = np.arange(min_coords[0], max_coords[0], spacing)
    y = np.arange(min_coords[1], max_coords[1], spacing)
    z = np.arange(min_coords[2], max_coords[2], spacing)

    # Generate all (x, y, z) combinations and reshape into an (N, 3) matrix
    grid = np.stack(np.meshgrid(x, y, z), axis=-1).reshape(-1, 3)

    print(f" Grid created: {grid.shape[0]:,} points at {spacing} Å spacing.")
    return grid


# --- Execution ---
if 'clean_atoms' in locals():
    grid = create_search_grid(clean_atoms, spacing=1.0)

 Grid created: 174,900 points at 1.0 Å spacing.


---
## 3. Filtering Pocket Candidates

The grid contains many irrelevant points. We keep only points near the protein surface using three successive distance filters.

### Filter 1 — Collision check (`min_dist = 2.0 Å`)

Points closer than 2.0 Å to any protein atom are inside the van-der-Waals radius and are discarded.

---

### Filter 2 — Surface proximity (`max_dist = 5.0 Å`)

Points with no protein atom within 5.0 Å are in bulk solvent and are removed.
This is intentionally stricter than the surface accessibility radius in Filter 3.

---

### Filter 3 — Surface accessibility check (`surface_threshold = 6.5 Å`)

To exclude points trapped deep inside internal cavities, we count how many protein atoms are within 6.5 Å of each remaining candidate.

* Points with **fewer than 25** neighbors are too isolated (noise or solvent) → removed
* Points with **more than 50** neighbors are buried too deeply inside the protein → removed
* Only points in the range **[25 – 50]** survive, representing surface-accessible pockets.

---

### Efficient search using KDTree

All three filters use a single KDTree built once over all atom coordinates, reducing the computation from O(N²) to **O(N log N)**.

---

### Result

The output is a reduced NumPy array of shape `(M, 3)` representing candidate binding pocket locations.

In [275]:
def find_pocket_points(
    grid_points: np.ndarray,
    protein_atoms: list,
    min_dist: float = 2.0,
    max_dist: float = 5.0,
    surface_threshold: float = 6.5,
    min_neighbors: int = 32,
    max_neighbors: int = 50
) -> np.ndarray:
    """
    Filters grid points down to potential binding pocket candidates.

    Three successive filters are applied:
        1. Collision check:       remove points closer than min_dist to any atom
        2. Surface proximity:     remove points with no atom within max_dist
        3. Surface accessibility: keep only points with atom count in
                                  [min_neighbors, max_neighbors] within surface_threshold

    Parameters:
        grid_points:       numpy array (N, 3) of all grid points
        protein_atoms:     list of Biopython Atom objects
        min_dist:          collision radius (Å) — points inside protein are removed
        max_dist:          surface proximity radius (Å) — bulk solvent points are removed
        surface_threshold: neighbor-count radius (Å) — used for accessibility filter
        min_neighbors:     minimum atom neighbors required (too few = isolated / solvent)
        max_neighbors:     maximum atom neighbors allowed (too many = buried inside protein)

    Returns:
        pocket_points: numpy array (M, 3) of surviving candidate points
    """
    if len(grid_points) == 0:
        return np.empty((0, 3))

    # Extract coordinates from Biopython objects
    atom_coords = np.array([atom.get_coord() for atom in protein_atoms])

    # Build the KDTree once — reused for all three filters
    tree = KDTree(atom_coords)

    # --- Filter 1: Collision check ---
    # If any atom is within min_dist, the point is inside the protein → discard
    clash_neighbors = tree.query_ball_point(grid_points, min_dist)
    no_clash_mask = np.array([len(n) == 0 for n in clash_neighbors])
    candidates = grid_points[no_clash_mask]

    if len(candidates) == 0:
        return np.empty((0, 3))

    # --- Filter 2: Surface proximity ---
    # Keep only points that have at least one atom within max_dist (not bulk solvent).
    # max_dist is intentionally smaller than surface_threshold so this filter is not redundant.
    surface_neighbors = tree.query_ball_point(candidates, max_dist)
    near_surface_mask = np.array([len(n) > 0 for n in surface_neighbors])
    candidates = candidates[near_surface_mask]

    if len(candidates) == 0:
        return np.empty((0, 3))

    # --- Filter 3: Surface accessibility ---
    # Count protein atoms within surface_threshold.
    # Points with too few neighbors are isolated (solvent); too many are buried.
    surface_access = tree.query_ball_point(candidates, surface_threshold)
    neighbor_counts = [len(n) for n in surface_access]

    print(f" Neighbor count in {surface_threshold} Å radius -> "
          f"Min: {min(neighbor_counts)}, Max: {max(neighbor_counts)}, "
          f"Mean: {sum(neighbor_counts)/len(neighbor_counts):.1f}")

    surface_access_mask = np.array([
        min_neighbors <= len(n) <= max_neighbors
        for n in surface_access
    ])
    pocket_points = candidates[surface_access_mask]

    print(f"Filtered: {len(grid_points):,} → {len(pocket_points):,} pocket candidates "
          f"({len(pocket_points)/len(grid_points)*100:.1f}% remaining).")
    return pocket_points


# --- Execution with runtime measurement ---
if 'clean_atoms' in locals() and 'grid' in locals():
    start = time.time()
    pocket_candidates = find_pocket_points(grid, clean_atoms)
    elapsed = time.time() - start
    print(f"Runtime: {elapsed:.1f}s")

 Neighbor count in 6.5 Å radius -> Min: 2, Max: 77, Mean: 20.7
Filtered: 174,900 → 5,009 pocket candidates (2.9% remaining).
Runtime: 0.6s


---
## 4. Cluster Analysis: Identifying Individual Pockets

The filtered grid points are grouped into clusters to identify discrete candidate binding regions.
Dense groups of neighboring points represent actual pockets; isolated points are noise.

### Why DBSCAN instead of k-Means?
- DBSCAN requires **no prior specification** of the number of clusters
- DBSCAN detects clusters of arbitrary shape (pockets are irregular)
- DBSCAN marks isolated outlier points as **noise** (label `-1`)

### Parameters
| Parameter | Meaning | Value used |
|-----------|---------|------------|
| `eps` | Maximum distance between two points in the same cluster | 1.5 Å |
| `min_samples` | Minimum points required to form a cluster | 15 |
| `MIN_POINTS` | Discard clusters smaller than this (too small to be a real pocket) | 60 |
| `MAX_POINTS` | Discard clusters larger than this (likely a poorly separated surface region) | 1000 |

In [276]:
def cluster_pocket_points(
    pocket_points: np.ndarray,
    eps: float = 1.2,
    min_samples: int = 20,
    MIN_POINTS: int = 60,
    MAX_POINTS: int = 1000
) -> dict:
    """
    Groups pocket candidate points into discrete binding pockets using DBSCAN.

    Parameters:
        pocket_points: numpy array (N, 3) of filtered candidate points
        eps:           maximum distance between two points in the same cluster (Å)
        min_samples:   minimum number of points required to form a valid cluster
        MIN_POINTS:    clusters smaller than this are discarded
        MAX_POINTS:    clusters larger than this are discarded

    Returns:
        pockets: dict {rank_id: np.ndarray of points}, sorted largest first,
                 with rank_ids as consecutive integers starting at 1
    """
    if len(pocket_points) == 0:
        print(" No points available for clustering.")
        return {}

    # Run DBSCAN clustering
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(pocket_points)

    # Group points by cluster label (exclude noise label -1)
    unique_labels = [label for label in set(labels) if label != -1]
    raw_pockets = {
        label: pocket_points[labels == label]
        for label in unique_labels
    }

    # Sort by cluster size descending (largest = most likely binding site)
    sorted_pockets = dict(
        sorted(raw_pockets.items(), key=lambda item: len(item[1]), reverse=True)
    )

    # Filter too small and too large clusters, then re-index from 1
    pockets = {
        new_id: pts
        for new_id, (_, pts) in enumerate(
            ((label, pts) for label, pts in sorted_pockets.items()
             if MIN_POINTS <= len(pts) <= MAX_POINTS),
            start=1
        )
    }

    # Print summary
    noise_count = int(np.sum(labels == -1))
    print(f"{len(pockets)} pockets found ({noise_count} noise points discarded):")

    for p_id, p_points in pockets.items():
        center = np.mean(p_points, axis=0)
        print(f"   Pocket {p_id}: {len(p_points):>4} points | Center: "
              f"({center[0]:.1f}, {center[1]:.1f}, {center[2]:.1f})")

    return pockets


# --- Execution ---
if 'pocket_candidates' in locals():
    pockets_dict = cluster_pocket_points(pocket_candidates)

0 pockets found (5009 noise points discarded):


---
## 5. Export for Visualization in PyMOL / UCSF Chimera

All pipeline stages are exported as **PDB files** so results can be inspected visually.
Each grid point is written as a **HETATM dummy atom**.

### Exported files

| File | Description |
| ----------------------------- | ------------------------------------ |
| `step1_full_grid.pdb` | Complete 3D search grid |
| `step2_pocket_candidates.pdb` | Filtered surface candidate points |
| `step3_pocket_N.pdb` | Individual clustered binding pockets |

### Why export?

* Enables **visual validation** of the algorithm
* Confirms pockets are located on the **protein surface**
* Provides output for **further analysis or docking**

**Workflow in PyMOL:**
1. `File → Open → 1H8D_cleaned.pdb`
2. `File → Open → step3_pocket_0.pdb` (repeat for each pocket)
3. Select pocket object: `as spheres`
4. Colour: `color red, pocket0` etc.

**Workflow in UCSF Chimera:**
1. `File → Open → 1H8D_cleaned.pdb`
2. `File → Open → step3_pocket_0.pdb`
3. `Actions → Atoms/Bonds → sphere`
4. `Actions → Color → ...`

In [277]:
def save_points_to_pdb(points: np.ndarray, output_file: str) -> None:
    """
    Saves a numpy array of 3D coordinates as a PDB file.
    Each point is written as a HETATM entry (dummy atom 'P').

    Atom serial numbers wrap at 99999 to stay within PDB column width.

    Parameters:
        points:      numpy array (N, 3) with XYZ coordinates
        output_file: target file path
    """
    with open(output_file, 'w') as f:
        for i, (x, y, z) in enumerate(points):
            # Wrap serial at 99999 to prevent column overflow in PDB format
            serial = (i % 99999) + 1
            f.write(
                f"HETATM{serial:5d}  P   PTS A   1    "
                f"{x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           P\n"
            )
        f.write("END\n")
    print(f" Saved: {output_file} ({len(points)} points)")


def export_all_steps(clean_atoms: list, output_dir: str = ".") -> None:
    """
    Runs the full geometry pipeline and exports each stage as a PDB file.

    Output files:
        step1_full_grid.pdb          — the complete grid around the protein
        step2_pocket_candidates.pdb  — filtered pocket candidate points
        step3_pocket_N.pdb           — individual clustered pockets (sorted by size)

    Parameters:
        clean_atoms: list of Biopython Atom objects (cleaned protein)
        output_dir:  directory in which all PDB files are saved
    """
    os.makedirs(output_dir, exist_ok=True)

    print(" Step 1: Generating grid ...")
    full_grid = create_search_grid(clean_atoms, spacing=1.0)
    save_points_to_pdb(full_grid, f"{output_dir}/step1_full_grid.pdb")

    print("\nStep 2: Filtering candidates ...")
    candidates = find_pocket_points(full_grid, clean_atoms)
    save_points_to_pdb(candidates, f"{output_dir}/step2_pocket_candidates.pdb")

    print("\n Step 3: Clustering pockets ...")
    pockets = cluster_pocket_points(candidates)

    print("\n Exporting pockets ...")
    for rank, (p_id, p_points) in enumerate(pockets.items()):
        save_points_to_pdb(p_points, f"{output_dir}/step3_pocket_{rank}.pdb")

    print(f"\nExport complete. {len(pockets)} pocket files saved.")


# --- Execution ---
# FIX: guard now checks 'clean_atoms', not 'atoms'
if 'clean_atoms' in locals():
    export_all_steps(clean_atoms, output_dir="pocket_output")

 Step 1: Generating grid ...
 Grid created: 174,900 points at 1.0 Å spacing.
 Saved: pocket_output/step1_full_grid.pdb (174900 points)

Step 2: Filtering candidates ...
 Neighbor count in 6.5 Å radius -> Min: 2, Max: 77, Mean: 20.7
Filtered: 174,900 → 5,009 pocket candidates (2.9% remaining).
 Saved: pocket_output/step2_pocket_candidates.pdb (5009 points)

 Step 3: Clustering pockets ...
0 pockets found (5009 noise points discarded):

 Exporting pockets ...

Export complete. 0 pocket files saved.


---
## 6. Residue Extraction — Identifying Pocket-Lining Residues

After identifying the pocket clusters, we determine **which amino acids form the pocket walls**.

For each pocket point, we search for nearby protein atoms using a **KDTree**.
If any atom of a residue lies within a distance threshold, the entire residue is considered part of the pocket.

### How it works
For every pocket point, we find all protein atoms within `threshold` (4.5 Å).
Any residue owning at least one such atom is considered a lining residue.

### Why 4.5 Å?

This distance captures typical ligand–protein interactions, including:

* Van der Waals contacts (~3.5 Å)
* Hydrogen bonds donor/acceptor pairs (~3.5–4.0 Å)
* Electrostatic interactions (~4.5 Å)

### Output

For each pocket, a text file lists:

* Pocket ID and number of grid points
* All lining residues in `NAME-ChainSeq` format (e.g., HIS-H195, GLU-H198)

This provides the biological context of each predicted binding site.

In [278]:
def extract_and_save_residues(
    pockets_dict: dict,
    protein_atoms: list,
    output_file: str = "pocket_residues.txt",
    threshold: float = 4.5
) -> None:
    """
    Identifies all protein residues lining each pocket and saves them to a text file.

    A residue is considered a pocket-lining residue if any of its atoms
    lies within `threshold` Å of any pocket point.

    Parameters:
        pockets_dict:  dictionary {pocket_id: np.ndarray of pocket points}
        protein_atoms: list of Biopython Atom objects
        output_file:   path to save the residue report
        threshold:     maximum distance from pocket point to atom in Å
    """
    # Build a KDTree over all protein atom coordinates for fast lookup
    atom_coords = np.array([atom.get_coord() for atom in protein_atoms])
    tree = KDTree(atom_coords)

    with open(output_file, 'w') as f:
        f.write("LIGAND BINDING SITE PREDICTIONS — RESIDUE LIST\n\n")

        for p_id, points in pockets_dict.items():
            # For every pocket point, find all atom indices within threshold
            neighbor_indices = tree.query_ball_point(points, threshold)

            # Flatten to a set of unique atom indices
            flat_indices = set(idx for sublist in neighbor_indices for idx in sublist)

            # Map atom indices → unique residue identifiers
            found_residues = set()
            for idx in flat_indices:
                res = protein_atoms[idx].get_parent()
                chain_id = res.get_parent().id
                hetflag, resseq, icode = res.id
                resnum = f"{resseq}{icode.strip()}"
                # Format: RESNAME-ChainResnum, e.g. HIS-H195
                res_info = f"{res.get_resname()}-{chain_id}{resnum}"
                found_residues.add(res_info)

            sorted_res = sorted(found_residues)

            f.write(f"Pocket {p_id} ({len(points)} points):\n")
            f.write(f"Lining Residues: {', '.join(sorted_res)}\n\n")

            print(f"   Pocket {p_id}: {len(sorted_res)} lining residues identified.")

    print(f"\nResidue report saved to: {output_file}")


# --- Execution ---
# FIX: guard checks 'clean_atoms', not 'atoms'
if 'clean_atoms' in locals():
    full_grid = create_search_grid(clean_atoms, spacing=1.0)
    pocket_candidates = find_pocket_points(full_grid, clean_atoms)
    pockets_dict = cluster_pocket_points(pocket_candidates)
    extract_and_save_residues(pockets_dict, clean_atoms)

 Grid created: 174,900 points at 1.0 Å spacing.
 Neighbor count in 6.5 Å radius -> Min: 2, Max: 77, Mean: 20.7
Filtered: 174,900 → 5,009 pocket candidates (2.9% remaining).
0 pockets found (5009 noise points discarded):

Residue report saved to: pocket_residues.txt


---
## 6b. Evolutionary Conservation Scoring

Binding pockets are not only geometric — they are also **evolutionarily conserved**.
A pocket lining that is invariant across related species is far more likely to be a true functional binding site.

### How conservation is calculated

A **Multiple Sequence Alignment (MSA)** of homologous proteins is read from a FASTA file.
The first sequence in the alignment is assumed to be the target protein matching the PDB structure.

For each alignment column, a conservation score between **0.0** and **1.0** is computed as:

```
score = count(most_common_residue) / count(all_sequences_in_alignment)
```

This penalises columns where many species have a gap (insertion/deletion) at that position.

These column-level scores are then mapped to individual PDB residue numbers by tracking
non-gap positions in the target sequence. Chain IDs are included in residue keys to
handle multi-chain structures correctly.

### Pocket filtering

Each predicted pocket is evaluated by computing the **average conservation score of its
lining residues** (atoms within `threshold_dist` Å of any pocket point).

Pockets whose average score falls below `min_score` are discarded.

| `min_score` | Interpretation |
| ----------- | -------------- |
| 0.0 | No filter (keep everything) |
| 0.45 | Recommended default — retains moderately conserved sites |
| 1.0 | Only perfectly conserved pockets (almost nothing survives) |

In [279]:
# 3-letter to 1-letter amino acid code lookup
AA_3_TO_1 = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}


def calculate_conservation_scores(
    alignment_file: str,
    protein_atoms: list,
    aln_format: str = "fasta"
) -> dict:
    """
    Reads a Multiple Sequence Alignment (MSA) and calculates a conservation score
    (0.0 to 1.0) for each amino acid position. Maps scores to PDB residue keys
    in the format 'ChainID_ResSeq' (e.g. 'H_50').

    The score for each column is:
        most_common_residue_count / total_sequences_in_alignment
    This penalises columns with many gaps across species.

    The first sequence in the alignment is assumed to be the target protein.

    Parameters:
        alignment_file: path to FASTA (or other format) MSA file
        protein_atoms:  list of Biopython Atom objects from the cleaned structure
        aln_format:     BioPython AlignIO format string (default: 'fasta')

    Returns:
        conservation_dict: {residue_key: score} e.g. {'H_50': 0.87, 'H_51': 0.42, ...}
    """
    print(f"Reading alignment from {alignment_file} ...")
    alignment = AlignIO.read(alignment_file, aln_format)
    target_seq_aligned = str(alignment[0].seq)
    num_sequences = len(alignment)

    # 1. Calculate conservation score for each alignment column
    col_scores = []
    for i in range(alignment.get_alignment_length()):
        column = alignment[:, i]
        residues_no_gap = [r.upper() for r in column if r not in ('-', '.')]

        if not residues_no_gap:
            col_scores.append(0.0)
            continue

        counts = Counter(residues_no_gap)
        most_common_count = counts.most_common(1)[0][1]
        # Denominator = all sequences (including those with a gap here)
        # This penalises insertion/deletion positions
        conservation_ratio = most_common_count / num_sequences
        col_scores.append(conservation_ratio)

    # 2. Build ordered list of PDB residue keys from atom list (chain-aware)
    pdb_residues = []
    full_pdb_seq = ""
    for atom in protein_atoms:
        res = atom.get_parent()
        chain_id = res.get_parent().id
        resseq = res.id[1]
        res_key = f"{chain_id}_{resseq}"

        if not pdb_residues or pdb_residues[-1] != res_key:
            pdb_residues.append(res_key)
            full_pdb_seq += AA_3_TO_1.get(res.get_resname(), "X")

    # 3. Find alignment offset (handles truncated or offset structures)
    aligned_seq_no_gaps = target_seq_aligned.replace('-', '').replace('.', '').upper()
    offset = full_pdb_seq.find(aligned_seq_no_gaps)

    if offset == -1:
        print("\n[WARNING] Alignment does not match structure exactly. Falling back to offset=0.")
        offset = 0
    else:
        print(f"\n[OK] Alignment offset found: starts at residue {offset + 1} in the 3D structure.")

    # 4. Map alignment columns to PDB residue keys
    conservation_dict = {}
    pdb_idx = offset

    for i, char in enumerate(target_seq_aligned):
        if char not in ('-', '.'):
            if pdb_idx < len(pdb_residues):
                res_key = pdb_residues[pdb_idx]
                conservation_dict[res_key] = col_scores[i]
                pdb_idx += 1

    print(f"Calculated conservation scores for {len(conservation_dict)} residues.")
    return conservation_dict


def filter_pockets_by_conservation(
    pockets_dict: dict,
    protein_atoms: list,
    conservation_dict: dict,
    threshold_dist: float = 4.5,
    min_score: float = 0.45
) -> dict:
    """
    Filters predicted pockets, retaining only those whose lining residues
    have an average conservation score >= min_score.

    Parameters:
        pockets_dict:      {pocket_id: np.ndarray of pocket points}
        protein_atoms:     list of Biopython Atom objects
        conservation_dict: {residue_key: score} from calculate_conservation_scores
        threshold_dist:    distance (Å) used to define lining residues
        min_score:         minimum average conservation score to retain a pocket.
                           Range 0.0–1.0. Recommended default: 0.45.

    Returns:
        conserved_pockets: filtered dict with new consecutive integer keys
    """
    print("\n--- Filtering Pockets by Evolutionary Conservation ---")

    atom_coords = np.array([atom.get_coord() for atom in protein_atoms])
    tree = KDTree(atom_coords)

    conserved_pockets = {}

    for p_id, points in pockets_dict.items():
        neighbor_indices = tree.query_ball_point(points, threshold_dist)
        flat_indices = set(idx for sublist in neighbor_indices for idx in sublist)

        # Collect unique residue keys (chain-aware)
        pocket_res_keys = set()
        for idx in flat_indices:
            res = protein_atoms[idx].get_parent()
            chain_id = res.get_parent().id
            resseq = res.id[1]
            pocket_res_keys.add(f"{chain_id}_{resseq}")

        scores = [conservation_dict.get(r_key, 0.0) for r_key in pocket_res_keys]

        if not scores:
            continue

        avg_score = sum(scores) / len(scores)

        if avg_score >= min_score:
            print(f"   [KEEP] Pocket {p_id:2d}: Avg Score = {avg_score:.2f} "
                  f"({len(pocket_res_keys)} residues)")
            conserved_pockets[p_id] = points
        else:
            print(f"   [DROP] Pocket {p_id:2d}: Avg Score = {avg_score:.2f} (Too low)")

    print(f"\nResult: {len(conserved_pockets)} out of {len(pockets_dict)} pockets "
          f"passed the conservation filter.")

    # Re-index remaining pockets consecutively from 1
    final_pockets = {i + 1: pts for i, pts in enumerate(conserved_pockets.values())}
    return final_pockets

---
## 7. Chemical Analysis & Pocket Ranking

Not all pockets are equally suitable for ligand binding. We analyze the **chemical environment**
of each pocket to determine what type of ligand is most likely to bind there.

### Chemical Classification of Amino Acids

Residues are divided into **five chemical groups** (consistent with the code below):

| Group                  | Residues                          | Properties                          |
| ---------------------- | --------------------------------- | ----------------------------------- |
| **Hydrophobic**        | ALA, VAL, LEU, ILE, MET, PHE, TRP | Favor non-polar, lipophilic ligands |
| **Polar (uncharged)**  | SER, THR, ASN, GLN, TYR           | Form hydrogen bonds                 |
| **Positively charged** | LYS, ARG, HIS                     | Attract negatively charged ligands  |
| **Negatively charged** | ASP, GLU                          | Attract positively charged ligands  |
| **Special cases**      | GLY, PRO, CYS                     | Structural roles; CYS can form disulfide bonds; PRO constrains backbone |

### Ranking Score

Each pocket receives a score:
```
score = size_score + chemistry_bonus
```
where `size_score = number of grid points` (volume proxy) and
`chemistry_bonus` weights charged and polar residues more heavily
(better interaction potential with drug-like molecules).

### Output

For each pocket: rank, score, preferred ligand type, chemical composition, and lining residues.

In [280]:
def analyze_and_rank_pockets(
    pockets_dict: dict,
    protein_atoms: list,
    threshold: float = 4.5
) -> list:
    """
    Chemical analysis and ranking of binding pockets.

    Each pocket is evaluated based on:
    - Pocket size (volume proxy = number of grid points)
    - Chemical composition of lining residues
    - Hydrophobic vs. polar vs. charged balance

    Residues are classified into 5 chemical groups:
        Hydrophobic | Polar (uncharged) | Positive | Negative | Special

    Parameters:
        pockets_dict:  {pocket_id: np.ndarray of pocket points}
        protein_atoms: list of Biopython Atom objects
        threshold:     distance (Å) to define lining residues

    Returns:
        ranked_pockets: list of dicts, sorted by score descending
    """
    # 5-group amino acid classification (consistent with Markdown table above)
    HYDROPHOBIC = {'ALA', 'VAL', 'LEU', 'ILE', 'MET', 'PHE', 'TRP'}
    POLAR       = {'SER', 'THR', 'ASN', 'GLN', 'TYR'}
    POSITIVE    = {'LYS', 'ARG', 'HIS'}
    NEGATIVE    = {'ASP', 'GLU'}
    SPECIAL     = {'GLY', 'PRO', 'CYS'}   # structural roles; PRO + CYS moved here

    # Build KDTree once over all protein atoms
    atom_coords = np.array([atom.get_coord() for atom in protein_atoms])
    tree = KDTree(atom_coords)

    results = []

    for p_id, points in pockets_dict.items():
        neighbor_indices = tree.query_ball_point(points, threshold)
        flat_indices = set(idx for sublist in neighbor_indices for idx in sublist)

        # Collect unique residue objects (not atoms) to avoid double-counting
        unique_residues = set()
        for idx in flat_indices:
            unique_residues.add(protein_atoms[idx].get_parent())

        residues = [res.get_resname() for res in unique_residues]

        # Count each chemical group
        counts = {
            'hydrophobic': sum(r in HYDROPHOBIC for r in residues),
            'polar':       sum(r in POLAR       for r in residues),
            'positive':    sum(r in POSITIVE    for r in residues),
            'negative':    sum(r in NEGATIVE    for r in residues),
            'special':     sum(r in SPECIAL     for r in residues),
        }

        total = sum(counts.values()) if sum(counts.values()) > 0 else 1

        hydrophobic_ratio = counts['hydrophobic'] / total
        polar_ratio       = counts['polar']       / total
        charge_ratio      = (counts['positive'] + counts['negative']) / total

        # Determine ligand type preference
        if charge_ratio > 0.30:
            preference = "Charged Ligands"
        elif hydrophobic_ratio > 0.50:
            preference = "Hydrophobic Ligands"
        elif polar_ratio > 0.30:
            preference = "Polar Ligands"
        else:
            preference = "Mixed Ligands"

        # Scoring: size + chemistry bonus (charged/polar residues weighted higher)
        size_score = len(points)
        chemistry_bonus = (
            counts['hydrophobic'] * 2 +
            counts['polar']       * 2 +
            counts['positive']    * 3 +
            counts['negative']    * 3
        )
        score = size_score + chemistry_bonus

        results.append({
            'id':               p_id,
            'size':             len(points),
            'score':            score,
            'preference':       preference,
            'composition':      counts,
            'hydrophobic_ratio': hydrophobic_ratio,
            'polar_ratio':      polar_ratio,
            'charge_ratio':     charge_ratio,
            'residues':         sorted(residues),
        })

    ranked_pockets = sorted(results, key=lambda x: x['score'], reverse=True)
    return ranked_pockets


if 'pockets_dict' in locals():
    ranked_list = analyze_and_rank_pockets(pockets_dict, clean_atoms)

    print("\n--- POCKET RANKING ---\n")

    with open("pocket_ranking.txt", "w") as f:
        f.write("BINDING SITE ANALYSIS\n\n")

        for i, p in enumerate(ranked_list):
            text = (
                f"Rank {i+1} | Pocket {p['id']} | Score: {p['score']}\n"
                f"Size: {p['size']} grid points\n"
                f"Preferred Ligand Type: {p['preference']}\n"
                f"Composition:\n"
                f"  Hydrophobic: {p['composition']['hydrophobic']}\n"
                f"  Polar:       {p['composition']['polar']}\n"
                f"  Positive:    {p['composition']['positive']}\n"
                f"  Negative:    {p['composition']['negative']}\n"
                f"  Special:     {p['composition']['special']}\n"
                f"Residues:\n"
                f"  {', '.join(p['residues'])}\n\n"
            )
            print(text)
            f.write(text)

    print("Saved to pocket_ranking.txt")


--- POCKET RANKING ---

Saved to pocket_ranking.txt


---
## 8. Validation Test — Checking the Analysis

Before trusting the results, we run a structured sanity check:

1. **Was a top pocket found?** → Confirms the pipeline produced at least one meaningful result
2. **Is the polarity ratio sensible?** → Extreme values (0.0 or 1.0) may indicate a data issue
3. **Are polar and non-polar pockets both represented?** → A healthy dataset should have both

This test cell is useful to re-run after changing parameters like `spacing`, `min_dist`, or `eps`.

In [281]:
def test_final_analysis(pockets_dict: dict, protein_atoms: list) -> None:
    """
    Runs a structured validation of the chemical analysis and ranking.
    Prints a summary of the top pocket and overall statistics.

    Parameters:
        pockets_dict:  dictionary {pocket_id: np.ndarray of pocket points}
        protein_atoms: list of Biopython Atom objects
    """
    print("Step 8: Validating Chemical Analysis & Ranking...")

    ranked_results = analyze_and_rank_pockets(pockets_dict, protein_atoms)

    if not ranked_results:
        print(" FAILED: No pockets available for analysis.")
        return

    # Check 1: Top pocket summary
    top = ranked_results[0]
    print(f"\n TOP PREDICTED BINDING SITE: Pocket {top['id']}")
    print(f"   Size (grid points): {top['size']}")
    print(f"   Best suited for:    {top['preference']}")
    print(f"   Polarity ratio:     {top['polar_ratio']:.1%} polar residues")
    print(f"   Key residues:       {', '.join(top['residues'][:8])}"
          f"{'...' if len(top['residues']) > 8 else ''}")

    # Check 2: Overall statistics
    hydrophobic_pockets = [int(p['id']) for p in ranked_results if p['preference'] == "Hydrophobic Ligands"]
    polar_pockets       = [int(p['id']) for p in ranked_results if p['preference'] == "Polar Ligands"]
    charged_pockets     = [int(p['id']) for p in ranked_results if p['preference'] == "Charged Ligands"]
    mixed_pockets       = [int(p['id']) for p in ranked_results if p['preference'] == "Mixed Ligands"]

    print(f"\n--- Summary Statistics ---")
    print(f"   Total pockets found:      {len(ranked_results)}")
    print(f"   Hydrophobic pockets:      {len(hydrophobic_pockets)}")
    print(f"   Polar pockets:            {len(polar_pockets)}")
    print(f"   Charged pockets:          {len(charged_pockets)}")
    print(f"   Mixed chemistry pockets:  {len(mixed_pockets)}")

    print("\nPocket IDs by category:")
    print(f"   Hydrophobic: {hydrophobic_pockets}")
    print(f"   Polar:       {polar_pockets}")
    print(f"   Charged:     {charged_pockets}")
    print(f"   Mixed:       {mixed_pockets}")

    # Check 3: Sanity check on top pocket
    if top['size'] > 0 and 0.0 < top['polar_ratio'] < 1.0:
        print("\nSUCCESS: Analysis looks biologically reasonable.")
    elif top['polar_ratio'] in (0.0, 1.0):
        print("\n WARNING: Extreme polarity ratio — check residue classification.")
    else:
        print("\nWARNING: Unexpected result — check input data.")


# --- Execution ---
if 'pockets_dict' in locals():
    test_final_analysis(pockets_dict, clean_atoms)

Step 8: Validating Chemical Analysis & Ranking...
 FAILED: No pockets available for analysis.


---
## 9. Full Pipeline with Conservation Filtering

`run_full_prediction` is the **master orchestration function** that chains all steps into a single call.

### Pipeline order

| Step | Function | Purpose |
| ---- | -------- | ------- |
| 1 | `create_search_grid` | Build uniform 3D grid around the protein |
| 2 | `find_pocket_points` | Filter grid to surface-accessible candidates |
| 3 | `cluster_pocket_points` | Group candidates into discrete pockets (DBSCAN) |
| 4 | `calculate_conservation_scores` | Derive per-residue conservation from MSA |
| 5 | `filter_pockets_by_conservation` | Discard pockets below the conservation threshold |
| 6 | `analyze_and_rank_pockets` | Chemical scoring and ranking of surviving pockets |
| 7 | `save_points_to_pdb` | Export conserved pockets directly (not a pipeline re-run) |

### Prerequisites

* `clean_atoms` must be available (output of Step 1c)
* An `alignment.fasta` MSA file must be present in the working directory

### Reading the Final Output

| Field | What it means |
|-------|---------------|
| `RANK 1` | Most likely binding site (highest score) |
| `Score` | Size + chemistry bonus (larger = more promising) |
| `Preferred Ligand Type` | Guides ligand selection strategy |
| `% polar` | Fraction of lining residues that are polar |
| `Key Residues` | First 8 unique amino acids lining the pocket |

> **Drug discovery context:** A high-score pocket with HIS, ASP, or GLU residues
> is a strong indicator of a catalytic site — ideal for competitive inhibitors.

In [282]:
def run_full_prediction(
    protein_atoms: list,
    alignment_file: str = None,
    spacing: float = 1.0,
    output_dir: str = "final_conserved_pockets"
) -> tuple:
    """
    Runs the complete binding pocket prediction pipeline end-to-end,
    including optional evolutionary conservation filtering.

    Parameters:
        protein_atoms:  list of Biopython Atom objects (cleaned protein)
        alignment_file: path to FASTA MSA file (optional — filter disabled if None)
        spacing:        grid spacing in Å (default 1.0)
        output_dir:     directory for exported PDB files

    Returns:
        ranked_results:     list of ranked pocket dicts
        conserved_pockets:  dict of pockets that passed conservation filter
        all_pockets:        dict of all geometry pockets before filtering
    """
    print("Starting Pipeline...\n")

    # 1. Grid generation
    full_grid = create_search_grid(protein_atoms, spacing=spacing)

    # 2. Grid filtering
    pocket_candidates = find_pocket_points(full_grid, protein_atoms)

    # 3. Clustering
    all_pockets = cluster_pocket_points(pocket_candidates, eps=1.5, min_samples=15)

    # 4. Conservation scoring + filtering (optional)
    if alignment_file and os.path.isfile(alignment_file):
        cons_scores = calculate_conservation_scores(alignment_file, protein_atoms)

        print("\n--- SANITY CHECK: First 10 residue conservation scores ---")
        for res_key, score in list(cons_scores.items())[:10]:
            print(f"  Residue {res_key}: Score = {score:.2f}")

        conserved_pockets = filter_pockets_by_conservation(
            all_pockets,
            protein_atoms,
            conservation_dict=cons_scores,
            threshold_dist=4.5,
            min_score=0.45
        )
    else:
        print("\n  WARNING: No alignment file provided or file not found.")
        print("   Conservation filter is DISABLED — all geometry pockets are used.\n")
        conserved_pockets = all_pockets

    if not conserved_pockets:
        print("No conserved pockets found. Consider lowering min_score.")
        return [], {}, all_pockets

    # 5. Chemical analysis on conserved pockets only
    ranked_results = analyze_and_rank_pockets(conserved_pockets, protein_atoms)

    # 6. Export conserved pockets
    os.makedirs(output_dir, exist_ok=True)
    for rank, (p_id, points) in enumerate(conserved_pockets.items()):
        save_points_to_pdb(points, f"{output_dir}/conserved_pocket_{rank}.pdb")
    print(f"\nExported {len(conserved_pockets)} conserved pocket(s) to '{output_dir}/'")

    return ranked_results, conserved_pockets, all_pockets


# --- Execution ---
if 'clean_atoms' in locals():
    final_ranked_pockets, final_pockets_dict, all_pockets_dict = run_full_prediction(
        clean_atoms, alignment_file="alignment.fasta"
    )

Starting Pipeline...

 Grid created: 174,900 points at 1.0 Å spacing.
 Neighbor count in 6.5 Å radius -> Min: 2, Max: 77, Mean: 20.7
Filtered: 174,900 → 5,009 pocket candidates (2.9% remaining).
14 pockets found (2585 noise points discarded):
   Pocket 1:  175 points | Center: (8.9, -8.5, 6.9)
   Pocket 2:  152 points | Center: (12.0, -13.1, 20.9)
   Pocket 3:  145 points | Center: (4.3, 5.6, 30.0)
   Pocket 4:  118 points | Center: (14.0, -1.0, 36.1)
   Pocket 5:   95 points | Center: (21.7, -13.2, 24.6)
   Pocket 6:   91 points | Center: (31.7, 4.8, 18.1)
   Pocket 7:   91 points | Center: (17.5, 9.0, 2.3)
   Pocket 8:   77 points | Center: (0.8, 5.2, 8.2)
   Pocket 9:   73 points | Center: (3.0, -17.1, 22.5)
   Pocket 10:   73 points | Center: (11.7, 14.2, 32.7)
   Pocket 11:   69 points | Center: (8.4, 13.3, 20.6)
   Pocket 12:   63 points | Center: (-0.9, -11.3, 18.9)
   Pocket 13:   63 points | Center: (19.7, 6.0, 34.8)
   Pocket 14:   63 points | Center: (24.2, 8.9, 30.2)
Readin

---
## 10. Combined Visualisation Export

`save_protein_with_colored_pockets` writes a **single PDB file** containing both the protein
and all predicted pocket points, making it easy to open one file and see everything.

### Encoding scheme

* Protein atoms are written as standard `ATOM` records.
* Each pocket's grid points are written as `HETATM` records with residue name `PKT`.
* Every pocket is assigned a **unique chain letter** using a sequential rank index —
  not the raw pocket ID — so chain letters are always consecutive and never collide
  with the protein chain.

### Usage 

```python
File → Open → protein_with_pockets.pdb
color red,   chain A   # pocket rank 1
color blue,  chain B   # pocket rank 2
color green, chain C   # pocket rank 3
```

In [283]:
POCKET_CHAIN_LETTERS = [c for c in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" if c != "H"]


def save_protein_with_colored_pockets(
    protein_atoms: list,
    pockets_dict: dict,
    output_file: str
) -> None:
    """
    Saves the protein and all predicted pockets in a single PDB file.
    Each pocket is written as a separate HETATM chain for easy coloring.
    Atom serial numbers wrap at 99999 to stay within PDB format limits.

    Parameters:
        protein_atoms: list of Biopython Atom objects
        pockets_dict:  {pocket_id: np.ndarray of pocket points}
        output_file:   path for the combined output .pdb file
    """
    with open(output_file, "w") as f:
        atom_id = 1

        # Write protein atoms
        for atom in protein_atoms:
            res   = atom.get_parent()
            chain = res.get_parent()
            x, y, z = atom.get_coord()
            serial = (atom_id % 99999) + 1

            f.write(
                f"ATOM  {serial:5d} {atom.get_name():>4s} "
                f"{res.get_resname():>3s} {chain.id:1s}"
                f"{res.id[1]:4d}    "
                f"{x:8.3f}{y:8.3f}{z:8.3f}"
                f"  1.00  0.00           C\n"
            )
            atom_id += 1

        # Write pocket points — one chain per pocket
        for rank, (pocket_id, points) in enumerate(pockets_dict.items()):
            chain_id = POCKET_CHAIN_LETTERS[rank % len(POCKET_CHAIN_LETTERS)]

            for point in points:
                x, y, z = point
                serial = (atom_id % 99999) + 1

                f.write(
                    f"HETATM{serial:5d}  P   PKT {chain_id}"
                    f"{rank + 1:4d}    "
                    f"{x:8.3f}{y:8.3f}{z:8.3f}"
                    f"  1.00  0.00           P\n"
                )
                atom_id += 1

        f.write("END\n")

    print("Saved combined file:", output_file)


# --- Execution ---
if 'clean_atoms' in locals() and 'all_pockets_dict' in locals():
    save_protein_with_colored_pockets(
        clean_atoms,
        all_pockets_dict,
        "protein_all_pockets.pdb"
    )
    save_protein_with_colored_pockets(
        clean_atoms,
        final_pockets_dict,
        "protein_conserved_pockets.pdb"
    )

Saved combined file: protein_all_pockets.pdb
Saved combined file: protein_conserved_pockets.pdb
